In [19]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob
from sklearn.metrics import classification_report
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

In [54]:
dataset=glob("dataset/*")
print(dataset)

['dataset/normal', 'dataset/tuberculosis']


In [21]:
X=[]
y=[]

for i in range(len(dataset)):
    imgs = glob(dataset[i] + "/*")

    for j in range(len(imgs)):
        img = cv2.imread(imgs[j])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (180, 180))

        X.append(img)
        y.append(i)


X = np.array(X)
y = np.array(y)
X = X.reshape(-1, 180, 180, 3)#.astype('float32') / 255.0

print(X.shape)
print(y.shape)
y = to_categorical(y)

(400, 180, 180, 3)
(400,)


In [22]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(2,activation = "softmax")   #Adding the Output Layer
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 178, 178, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 89, 89, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 87, 87, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 43, 43, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 41, 41, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 20, 20, 64)       

In [49]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2)
print(y_train.shape)

(320, 2)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])
model.fit(X_train,y_train,epochs=10)

In [51]:
y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred,axis=1)
print(y_pred.shape)
y_test=np.argmax(y_test,axis=1)
print(y_test.shape)

3/3 [==============================] - 3s 773ms/step
(80,)
(80,)


In [53]:
print(f"Accuracy using CNN : {100*accuracy_score(y_test,y_pred):.2f}%\n\n")
print(classification_report(y_pred,y_test))
# model.save("model_local.h5")

Accuracy using CNN : 97.50%


              precision    recall  f1-score   support

           0       0.95      1.00      0.97        38
           1       1.00      0.95      0.98        42

    accuracy                           0.97        80
   macro avg       0.97      0.98      0.97        80
weighted avg       0.98      0.97      0.98        80

